In [1]:
# import matplotlib.pyplot as plt
# import LovelyPlots.utils as lp
# import pandas as pd
import seaborn as sns
# plt.style.use(['ipynb','colors5-light'])
# lp.set_font('timesnewroman.ttf')

import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rc('xtick',labelsize=14)
rc('ytick',labelsize=14)

# Set the figure size
plt.clf()
plt.figure(constrained_layout=True)
plt.figure(figsize=(5, 4))

colors = ['#78C850',  '#705898', '#F85888','#6890F0','#F08030' ,'#F8D030', '#98D8D8']


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 500x400 with 0 Axes>

In [2]:
import pandas as pd 
import wandb
import numpy as np
api = wandb.Api()
entity, project = "johndoe", "bb_retrieval"  # set to your entity and project 

runs = api.runs(entity + "/" + project) 

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
         if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

# runs_df.to_csv("project.csv")

In [3]:
runs_df.head(1)

,summary,config,name
0,"{'_timestamp': 1685065605.9348073, 'MRR': 0.69...","{'gpu': 0, 'arch': 'swin2_base_384', 'rank': 0...",midas-swin2_base_384


In [4]:
get_feat_map = {
    'config' : ['pt_style','arch','dataset','featshape'],
    'summary': ['map','MRR','prs','recalls'],
    
}

def getcsv(fmap,df):
    csv_dict = {}
    for k,vlist in fmap.items():
        print(k,vlist)
        if vlist == 0:
            csv_dict[k] = df[k].tolist()
        else:
            temp = pd.DataFrame.from_records(df[k])
            temp = temp[vlist]
            td = temp.to_dict(orient='dict')
            csv_dict = csv_dict | td 

    newdf = pd.DataFrame.from_dict(csv_dict)

    return newdf

In [16]:
df = getcsv(get_feat_map,runs_df)
df['pt_arch'] = df['pt_style'] + '-' + df['arch']
df['recall@5'] = df['recalls'].apply(lambda x: x[1])
df.head()

config ['pt_style', 'arch', 'dataset', 'featshape']
summary ['map', 'MRR', 'prs', 'recalls']


,pt_style,arch,dataset,featshape,map,MRR,prs,recalls,pt_arch,recall@5
0,midas,swin2_base_384,roxford5k,1000,27.280509,0.690718,"[0.6571428571428571, 0.5142857142857142, 0.441...","[0.03929331969636929, 0.09770828136690413, 0.1...",midas-swin2_base_384,0.097708
1,midas,swin2_base_384,INSTRE,1000,24.098993,0.694229,"[0.608, 0.4932, 0.4269000000000005]","[0.01520000000000015, 0.06165, 0.1067250000000...",midas-swin2_base_384,0.061650
2,sscd,resnet50_disc,Objectnet,1024,0.834613,0.106283,"[0.056658130601792575, 0.03245838668373901, 0....","[0.0014206899148904103, 0.004066137765392141, ...",sscd-resnet50_disc,0.004066
3,sscd,resnet50_im,Objectnet,512,0.791882,0.101321,"[0.05025608194622279, 0.03156209987195923, 0.0...","[0.0012606386985011642, 0.003955514130534868, ...",sscd-resnet50_im,0.003956
4,midas,swin2_base_384,Objectnet,1000,1.895543,0.168034,"[0.0973111395646607, 0.06472471190781008, 0.05...","[0.002437381269195698, 0.008111301498832515, 0...",midas-swin2_base_384,0.008111


In [13]:
table = pd.pivot_table(df, values='map', index=['pt_arch'],
                       columns=['dataset'], aggfunc=np.mean)
table = table[['CUB200', 'iNat','Objectnet', 'INSTRE',  'google_landmark','roxford5k', 'rparis6k' ,'Copydays']]
table.to_csv('./results_pivot_may8_map.csv')

In [14]:
table = pd.pivot_table(df, values='MRR', index=['pt_arch'],
                       columns=['dataset'], aggfunc=np.mean)
table = table[['CUB200', 'iNat','Objectnet', 'INSTRE',  'google_landmark','roxford5k', 'rparis6k' ,'Copydays']]
table.to_csv('./results_pivot_may8_mrr.csv')

In [ ]:
table = pd.pivot_table(df, values='recall@5', index=['pt_arch'],
                       columns=['dataset'], aggfunc=np.mean)
table = table[['CUB200', 'iNat','Objectnet', 'INSTRE',  'google_landmark','roxford5k', 'rparis6k' ,'Copydays']]
table.to_csv('./results_pivot_may8_recall5.csv')

In [15]:
table

dataset,CUB200,iNat,Objectnet,INSTRE,google_landmark,roxford5k,rparis6k,Copydays
pt_arch,,,,,,,,
clip-resnet101,0.666829,0.149322,0.310017,0.926586,0.441399,0.646899,0.970476,0.862039
clip-resnet50,0.590587,0.117622,0.245123,0.875956,0.406701,0.668122,0.969048,0.861346
clip-resnet50_64,0.815592,0.259485,0.479749,0.972293,0.598963,0.792471,0.985714,0.932221
clip-vit_base_patch16_224,0.821622,0.232441,0.461357,0.981148,0.554476,0.865343,0.973810,0.929986
clip-vit_large_patch14_224,0.845913,0.261466,0.466923,0.978212,0.582241,0.822452,0.969048,0.944148
mae-vit_base_patch16_224,0.143100,0.010584,0.035467,0.339574,0.071637,0.330655,0.808489,0.611760
mae-vit_large_patch16_224,0.237556,0.036066,0.052821,0.555352,0.161384,0.532008,0.924975,0.812901
midas-swin2_base_384,0.509650,0.125555,0.168034,0.694229,0.360502,0.690718,0.969048,0.891385
midas-swin2_tiny_256,0.442582,0.076665,0.140106,0.634969,0.326553,0.586653,0.933983,0.864945
